**Tutorial for imblearn**

@ Date: 2025-03-23<br>
@ Author: Rui Zhu<br>
@ Follow: https://imbalanced-learn.org/stable/user_guide.html

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter

---
# 超采样(Over-sampling)

In [2]:
# 生成测试数据
from sklearn.datasets import make_classification
X, y = make_classification(
    n_samples=5000, n_features=2, n_informative=2, 
    n_redundant=0, n_repeated=0, n_classes=3, 
    n_clusters_per_class=1, 
    weights=[0.01, 0.05, 0.94], 
    class_sep=0.8, random_state=0
)
print(sorted(Counter(y).items()))

[(0, 64), (1, 262), (2, 4674)]


## 随机超采样
- 随机复制少数类样本，直到其数量与多数类相当或达到设定比例。
- 复制过程通常是有放回采样，即同一个样本可以被多次选中

In [3]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

print(sorted(Counter(y_resampled).items()))

[(0, 4674), (1, 4674), (2, 4674)]


In [4]:
import numpy as np
# 也可以超采样字符串数据
X_hetero = np.array([['xxx', 1, 1.0], ['yyy', 2, 2.0], ['zzz', 3, 3.0]],
                    dtype=object)
y_hetero = np.array([0, 0, 1])
X_resampled, y_resampled = ros.fit_resample(X_hetero, y_hetero)
print(X_resampled)
print(y_resampled)

[['xxx' 1 1.0]
 ['yyy' 2 2.0]
 ['zzz' 3 3.0]
 ['zzz' 3 3.0]]
[0 0 1 1]


In [5]:
from sklearn.datasets import fetch_openml
df_adult, y_adult = fetch_openml(
    'adult', version=2, as_frame=True, return_X_y=True)
df_adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States


In [6]:
df_resampled, y_resampled = ros.fit_resample(df_adult, y_adult)
df_resampled.head()  

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States


## SMOTE超采样
- SMOTE: the Syntheitc Minority Oversampling Technique
- 方法:
    1. 选取一个少数类样本，找到其k 个最近邻（通常 k=5）。
    2. 随机选择一个邻居，在它们之间的特征空间内进行线性插值，生成新的样本。
    3. 重复上述步骤，直到少数类样本数量达到目标比例。

In [7]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 4674), (1, 4674), (2, 4674)]


## ADASYN超采样
1. 计算少数类样本的密度分布：
    对于每个少数类样本，计算其 k 近邻 中多数类样本的占比，衡量其“难学”程度。
2. 确定采样权重：
    赋予多数类附近的少数类样本更高的采样权重，以生成更多新样本。
3. 合成新样本：
    使用与 SMOTE 类似的线性插值方法生成新样本，但采样数量因样本的分布情况而异。

In [8]:
from imblearn.over_sampling import ADASYN

X_resampled, y_resampled = ADASYN().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 4673), (1, 4662), (2, 4674)]


## SMOTE的变种
- BorderlineSMOTE
    1. 仅对靠近决策边界的少数类样本进行超采样，而不是整个少数类。
    2. 计算每个少数类样本的 k 近邻，如果大部分邻居是多数类，则认为该样本处于边界区域。
    3. 只对这些靠近多数类的边界样本进行 SMOTE 过采样。
- SVMSMOTE
    1. 结合SVM 分类器，找出支持向量（Support Vectors），并在这些样本附近生成新的少数类样本。
	2. 使用 SVM 找到少数类的支持向量（即靠近决策边界的样本）。
	3. 仅在支持向量周围进行 SMOTE 过采样，生成新的少数类样本。
- KMeansSMOTE
    1. 先对少数类样本进行 K-Means 聚类，然后在每个簇内使用 SMOTE 生成新样本。
    2. 对少数类样本进行 K-Means 聚类，划分成多个子群。
    3. 在每个簇内部进行 SMOTE 过采样，确保新样本的分布符合原始数据结构。

In [9]:
from imblearn.over_sampling import BorderlineSMOTE
X_resampled, y_resampled = BorderlineSMOTE().fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 4674), (1, 4674), (2, 4674)]


## SMOTENC处理‘连续’和‘类别’混合特征
- 以上各种方法均不能处理类别特征
- 当特征中出现非连续的类别特征时, 应使用SMOTENC

In [10]:
# create a synthetic data set with continuous and categorical features
rng = np.random.RandomState(42)
n_samples = 50
X = np.empty((n_samples, 3), dtype=object)
X[:, 0] = rng.choice(['A', 'B', 'C'], size=n_samples).astype(object)
X[:, 1] = rng.randn(n_samples)
X[:, 2] = rng.randint(3, size=n_samples)
y = np.array([0] * 20 + [1] * 30)
print(sorted(Counter(y).items()))

pd.DataFrame(X, columns=['X0', 'X1', 'X2']).head()

[(0, 20), (1, 30)]


,X0,X1,X2
0,C,-0.140218,2
1,A,-0.033193,2
2,C,-0.749077,1
3,C,-0.778382,2
4,A,0.948843,2


In [11]:
from imblearn.over_sampling import SMOTENC

smote_nc = SMOTENC(categorical_features=[0, 2], random_state=0)

X_resampled, y_resampled = smote_nc.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))
print(X_resampled[-5:])

[(0, 30), (1, 30)]
[['A' 0.19899937789791136 2]
 ['B' -0.3657680728116921 2]
 ['B' 0.879082872958526 2]
 ['B' 0.371089161882461 2]
 ['B' 0.3327240726719727 2]]


## SMOTENE处理只有‘不连续的’特征

In [12]:
import numpy as np
X = np.array(["green"] * 5 + ["red"] * 10 + ["blue"] * 7,
             dtype=object).reshape(-1, 1)
y = np.array(["apple"] * 5 + ["not apple"] * 3 + ["apple"] * 7 +
             ["not apple"] * 5 + ["apple"] * 2, dtype=object)

pd.DataFrame(np.hstack((X, y.reshape(-1, 1))), columns=['X', 'y']).head(10)

,X,y
0,green,apple
1,green,apple
2,green,apple
3,green,apple
4,green,apple
5,red,not apple
6,red,not apple
7,red,not apple
8,red,apple
9,red,apple


In [13]:
from imblearn.over_sampling import SMOTEN

sampler = SMOTEN(random_state=0)

X_res, y_res = sampler.fit_resample(X, y)

pd.DataFrame(np.hstack((X_res, y_res.reshape(-1, 1))),
             columns=['X_res', 'y_res']).head(10)

,X_res,y_res
0,green,apple
1,green,apple
2,green,apple
3,green,apple
4,green,apple
5,red,not apple
6,red,not apple
7,red,not apple
8,red,apple
9,red,apple


---
# 欠采样(Under-sampling)
- 欠采样方法可以分成2类:
    1. 控制样本数量的欠采样, 如随机欠采样
    2. 数据清洗欠采样, 清洗噪声, 或清理掉太容易分类的数据
- Tomek’s Link:
    1. 指的是两个最近邻样本分别来自不同类别
    2. 识别并删除 Tomek’s Link，从而清理数据集。
    3. Tomek’s Link 通常是分类困难或噪声点，去除它们有助于优化分类边界。

In [14]:
from collections import Counter
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=3,
                           n_clusters_per_class=1,
                           weights=[0.01, 0.05, 0.94],
                           class_sep=0.8, random_state=0)
print(sorted(Counter(y).items()))

[(0, 64), (1, 262), (2, 4674)]


## 原型生成(Prototype Generation)
- 减少目标类别的样本数量，但保留的样本是重新生成的，而非从原始数据集中直接选择的
- ClusterCentroids 方法：
    1. 使用 K-means 聚类来减少样本数量。
    2. 每个类别的样本会被替换为 K-means 计算得到的聚类中心，而不是原始样本。

In [ ]:
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=0)
X_resampled, y_resampled = cc.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 64), (2, 64)]


## 随机欠采样
- 随机选择目标类别中的一部分数据

In [16]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 64), (2, 64)]


In [17]:
# 随机欠采样允许有放回地采样, 即bootstrap
rus = RandomUnderSampler(random_state=0, replacement=True)
X_resampled, y_resampled = rus.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 64), (2, 64)]


## NearMiss (近邻删除)
- NearMiss有3个version, 对应不同的规则:
    1. version1: 扔掉与少数类最近的N个样本
    2. version2: 计算每个多数类样本到 k 个最远的少数类样本 的平均距离，扔掉距离最小的多数类样本
    3. 两步选择过程：
	    - 第一步：为每个少数类样本选择其最近的多数类邻居。
	    - 第二步：从这些邻居中选择到少数类样本平均距离最大的多数类样本进行保留。

In [18]:
from imblearn.under_sampling import NearMiss
nm1 = NearMiss(version=1)
X_resampled_nm1, y_resampled = nm1.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 64), (2, 64)]


## 最近邻编辑(EditedNearestNeighbours)
- 目标：通过最近邻方法编辑数据，删除“噪声”样本。
- 步骤：
    1. 使用 K-Nearest Neighbors（KNN）训练整个数据集。
    2. 对每个目标类样本，找到其 K 个最近邻（仅限目标类样本）。
    3. 如果目标样本的邻居大多数来自不同类别，则将其删除。
- 参数：
    1. kind_sel: 控制选择行为。'mode' 保留大多数邻居相同类的样本，'all' 会更具侵略性，删除更多样本。
    2. n_neighbors: 设定用于编辑的数据邻居数

In [19]:
sorted(Counter(y).items())
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours()
X_resampled, y_resampled = enn.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 213), (2, 4568)]


## 重复最近邻编辑(Repeated Edited Nearest Neighbours)
- 重复执行 ENN 算法，多次删除更多噪声样本。
- 停止条件：
	1.	达到最大迭代次数。
	2.	没有更多样本被删除。
	3.	多数类变为少数类。
	4.	在欠采样过程中，某个多数类完全消失。

In [20]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
renn = RepeatedEditedNearestNeighbours()
X_resampled, y_resampled = renn.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 208), (2, 4551)]


## All KNN
- 扩展：每轮基于更多邻居进行清洗，从 1-NN 开始，逐步增加邻居数。
- 停止条件：
	1. 达到最大邻居数。
	2. 多数类变为少数类。

In [21]:
from imblearn.under_sampling import AllKNN
allknn = AllKNN()
X_resampled, y_resampled = allknn.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 220), (2, 4601)]


## 浓缩最近邻(Condensed Nearst Neighbors)
1. 从目标类（少数类）选择一个样本，并将其与该类的其他样本一起添加到数据集 S。
2. 使用 1-NN 方法训练一个分类器，并对训练集中每个样本进行评估。
3. 如果样本被误分类，则将其移除；如果样本正确分类，则将其保留。
4. 重复这个过程直到没有误分类样本。
5. 返回最终的数据集 S，即已浓缩的数据集。

In [22]:
from imblearn.under_sampling import CondensedNearestNeighbour
cnn = CondensedNearestNeighbour(random_state=0)
X_resampled, y_resampled = cnn.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 24), (2, 115)]


## One Sided Selection
1. 选择少数类样本，并训练一个 1-NN 分类器。
2. 对训练集中的每个样本进行分类，找出误分类的样本。
3. 使用 Tomek Links 算法清除噪声样本。
4. 重复步骤，直到没有误分类或没有更多的噪声样本需要去除。
5. 返回最终的清理后的数据集。

In [23]:
from imblearn.under_sampling import OneSidedSelection
oss = OneSidedSelection(random_state=0)
X_resampled, y_resampled = oss.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 174), (2, 4404)]


## Neighbourhood Cleaning Rule
Neighbourhood Cleaning Rule (NCR) 是一种用于处理不平衡数据集和噪声样本的欠采样技术，旨在清理数据中的冗余和噪声样本。NCR 基于 K-Nearest Neighbors (KNN) 算法，通过检查样本在其邻域中的分布来决定是否需要删除样本。它结合了多种数据清理方法，特别是 Edited Nearest Neighbours (ENN) 和 Condensed Nearest Neighbours (CNN)，通过清理邻域中的噪声样本来优化数据集，改善分类器的性能。

In [24]:
from imblearn.under_sampling import NeighbourhoodCleaningRule
ncr = NeighbourhoodCleaningRule(n_neighbors=11)
X_resampled, y_resampled = ncr.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 193), (2, 4535)]


## Instance Hardness Threshold
通过删除那些难以正确分类的实例（硬实例）来提高分类器的性能

In [25]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import InstanceHardnessThreshold

iht = InstanceHardnessThreshold(random_state=0,
                                estimator=LogisticRegression())

X_resampled, y_resampled = iht.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 64), (1, 64), (2, 64)]


---
# 组合超采样和欠采样

In [27]:
from collections import Counter
from sklearn.datasets import make_classification
X, y = make_classification(n_samples=5000, n_features=2, n_informative=2,
                           n_redundant=0, n_repeated=0, n_classes=3,
                           n_clusters_per_class=1,
                           weights=[0.01, 0.05, 0.94],
                           class_sep=0.8, random_state=0)
print(sorted(Counter(y).items()))

[(0, 64), (1, 262), (2, 4674)]


In [28]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 4060), (1, 4381), (2, 3502)]


In [29]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)
print(sorted(Counter(y_resampled).items()))

[(0, 4499), (1, 4566), (2, 4413)]
